In [43]:
import heapq as h
import numpy as np

POISSON_PROCESS_NEW_REQUESTS_LAMBDA = 100
PARETO_PROCESS_NEW_FILE_SIZE_A = 1
PARETO_PROCESS_FILE_POPULARITY_A = 1
LOGNORMAL_PROCESS_ARRIVE_AT_QUEUE_MEAN = 0.5
LOGNORMAL_PROCESS_ARRIVE_AT_QUEUE_SIGMA = 0.4

TOTAL_NO_OF_FILES = 1000
INSTITUTIONAL_BANDWIDTH = 1000
ACCESS_LINK_BANDWIDTH = 15
TOTAL_TIME_TO_RUN = 100
CACHE_CAPACITY = 10
CACHE_MAX_ALLOWED_FILE_SIZE = 10


class Files:
    def __init__(self):
        self.popularity = np.random.pareto(PARETO_PROCESS_FILE_POPULARITY_A,size=TOTAL_NO_OF_FILES)
        self.popularity = self.popularity/np.sum(self.popularity)
        self.size = np.random.pareto(PARETO_PROCESS_NEW_FILE_SIZE_A,size=TOTAL_NO_OF_FILES)



class Simulation_Q():
    def __init__(self):
        self.q = []
    
    def push(self, event_tuple:tuple):
        h.heappush(self.q,event_tuple)

    def pop(self):
        return h.heappop(self.q)[1]
        
class Cache():
    def __init__(self,
    all_files:Files,
    init_file_list=[]):

        self.capacity = CACHE_CAPACITY
        self.store = {}
        self.storage_left = CACHE_CAPACITY
        self.all_files = all_files
        self.__subclass_declarations__()
        for i in range(len(init_file_list)):
            if not self.add_file(init_file_list[i]):
                break
    
    def __subclass_declarations__(self):
        pass

    def file_present(self,file_index):
        if file_index in self.store:
            return True
        return False


    def add_file(self,file_index):
        if file_index not in self.store:
            if self.storage_left > self.all_files.size[file_index]:
                self.store[file_index] = self.all_files.size[file_index]
                self.storage_left -= self.all_files.size[file_index]
            else:
                return False
        return True

# class LRU_cache(Cache):

#     def __subclass_declerations__(self):
#         self.use_heap = []
#         self.hit_counter = 0


#     def add_file()

class LRU_File():
    def __init__(self,file_index,file_size):
        self.next = None
        self.prev = None
        self.id = file_index
        self.size = file_size
        self.hits = 0


class LRU_Cache(Cache):
    
    def __subclass_declarations__(self):
        self.lru_root = None
        self.lru_tail = None
        self.hit_counter = 0

    def file_present(self,file_index):
        if file_index in self.store:
            print(file_index,'BR',self.get_lru_list())
            file = self.store[file_index]
            # self.hit_counter += 1
            if file.prev is not None and file.next is not None:
                file.prev.next = file.next
                self.lru_tail.next = file
                file.prev = self.lru_tail   
                self.lru_tail = file
                self.lru_tail.next = None
            elif file.prev is None and file.next is not None:
                self.lru_root = file.next
                self.lru_root.prev = None
                self.lru_tail.next = file
                file.prev = self.lru_tail   
                self.lru_tail = file
                self.lru_tail.next = None
            file.hits += 1
            
            print(file_index,'R',self.get_lru_list())
            return True
        return False


    def add_file(self,file_index,increment_counter=True):    
        if self.all_files.size[file_index] > CACHE_MAX_ALLOWED_FILE_SIZE or self.all_files.size[file_index] > CACHE_CAPACITY:
            return False  
        if file_index not in self.store:
            file = LRU_File(file_index,self.all_files.size[file_index]) 
            # if increment_counter:
            #     self.hit_counter += 1
            if self.storage_left > self.all_files.size[file_index]:

                if self.lru_root is None:
                    self.lru_root = file
                if self.lru_tail is None:
                    self.lru_tail = file
                else:
                    self.lru_tail.next = file
                    file.prev = self.lru_tail                    
                    self.lru_tail = file
                    self.lru_tail.next = None
                self.store[file.id] = file
                self.storage_left -= file.size
                file.hits += 1

            else:
                while(self.storage_left <= file.size):
                    if self.lru_root is None:
                        raise Exception('LRU Cache - Storage inconsistency')                       

                    file_to_discard = self.lru_root
                    root = file_to_discard.next
                    root.prev = None
                    del self.store[file_to_discard.id]
                    self.storage_left += file_to_discard.size
                self.add_file(file_index)

            print(file_index,'W',self.get_lru_list())  
        else:
            self.file_present(self,file_index)        
        
        return True
        

    def get_stored_file_list(self):
        file_list = []
        for key in self.store:
            file_list.append([self.store[key].id,self.store[key].hits])
        return file_list

    def get_lru_list(self):
        file_list = []
        node = self.lru_root
        while(node is not None):
            file_list.append(node.id)
            node = node.next
        return file_list


    # def add_file(self,file_index,increment_counter=True):    
    #     if self.all_files.size[file_index] > CACHE_MAX_ALLOWED_FILE_SIZE or self.all_files.size[file_index] > CACHE_CAPACITY:
    #         return False  
    #     if file_index not in self.store:            
    #         if increment_counter:
    #             self.hit_counter += 1
    #         if self.storage_left > self.all_files.size[file_index]:
    #             file_tuple = [self.hit_counter,self.all_files.size[file_index],file_index]
    #             self.store[file_index] = file_tuple
    #             h.heappush(self.lru_heap,file_tuple)
    #             self.storage_left -= self.all_files.size[file_index]
    #         else:
    #             while(self.storage_left <= self.all_files.size[file_index]):
    #                 file_tuple = h.heappop(self.lru_heap)                    
    #                 del self.store[file_tuple[2]]
    #                 self.storage_left += self.all_files.size[file_tuple[2]]
    #             self.all_files(file_index,increment_counter=False)  
    #     else:
    #         self.file_present(self,file_index)
    #     return True
        
            



class Simulator_Env():
    def __init__(self, Files:Files, Cache:Cache, cache_init_files=[]):

        self.sim_q = Simulation_Q()
        self.files = Files
        self.cache = Cache
        self.fifo = []
        self.log = []
        self.req_count = 0
    
    def get_total_times_for_reqs(self):
        return(np.array(self.log)[:,0])


class Event:

    def __init__(self, sim: Simulator_Env, create_time: int, parent:object=None):
        self.sim = sim
        self.create_time = create_time
        self.process_time = create_time
        self.parent = parent
        self.name = 'Event'
        self.__enqueue__()

    def get_super_parent(self):
        node = self
        while(node.parent is not None):
            node = node.parent
        return node

    def __lt__(self,any):
        return True

    def __gt__(self,any):
        return True

    def __enqueue__(self):
        pass


    def process(self):
        pass


class E_get_new_reqs(Event):

    def __enqueue__(self):
        self.name = 'Get New Requests'
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        reqs_to_handle = np.random.poisson(POISSON_PROCESS_NEW_REQUESTS_LAMBDA)
        for i in range(int(reqs_to_handle)):
            E_new_req(self.sim, self.process_time)


class E_new_req(Event):
    def __enqueue__(self):
        self.name = 'New Request'
        self.sim.req_count += 1
        self.file_index = np.argmax(np.random.multinomial(1,self.sim.files.popularity))
        self.file_size = self.sim.files.size[self.file_index]        
        # self.process_time = self.create_time + (self.file_size/INSTITUTIONAL_BANDWIDTH)
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        if sim.cache.file_present(self.file_index):
            E_file_recieved(self.sim,self.process_time,self)
        else:
            E_arrive_at_queue(self.sim,self.process_time,self)


class E_file_recieved(Event):
    def __enqueue__(self):
        self.name = 'File recieved'
        initial_req = self.get_super_parent()
        file_size = self.sim.files.size[initial_req.file_index]
        self.process_time = self.create_time + (file_size/ INSTITUTIONAL_BANDWIDTH)
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        initial_req = self.get_super_parent()
        log_data = [self.process_time - initial_req.create_time, initial_req.file_index, initial_req.file_size,self]
        self.sim.log.append(log_data)


class E_arrive_at_queue(Event):
    def __enqueue__(self):
        self.name = 'Arrive at queue'
        self.process_time = self.create_time + np.random.lognormal(LOGNORMAL_PROCESS_ARRIVE_AT_QUEUE_MEAN,
                                                                    LOGNORMAL_PROCESS_ARRIVE_AT_QUEUE_SIGMA)
        
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        if len(self.sim.fifo):
            self.sim.fifo.append(self)
        else:
            E_depart_from_queue(self.sim,self.process_time,self)


class E_depart_from_queue(Event):
    def __enqueue__(self):
        self.name = 'Depart from queue'
        initial_req = self.get_super_parent()
        self.process_time = self.create_time + (initial_req.file_size/ACCESS_LINK_BANDWIDTH)
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        initial_req = self.get_super_parent()
        sim.cache.add_file(initial_req.file_index)
        E_file_recieved(self.sim,self.process_time,self)
        if len(sim.fifo):
            event = sim.fifo.pop(0)
            E_depart_from_queue(self.sim,self.process_time,event)




In [44]:
# initialize simulator environment
np.random.seed(11)
files = Files()
cache = LRU_Cache(files)
# cache = Cache(files)
sim = Simulator_Env(files,cache)

#initialize new req events to be processed at every second
for i in range(TOTAL_TIME_TO_RUN):
    E_get_new_reqs(sim,i)


#Main simulator loop
while(len(sim.sim_q.q)):

    e = sim.sim_q.pop()
    e.process()
    
    

print(len(sim.log) == sim.req_count)

#Show logs
for data in sim.log:    
    e = data[3]
    path = []
    while(e is not None):
        path.insert(0,e.name)
        e = e.parent
    print('file name - ' + str(data[1]))    
    print('file size - ' + str(data[2]))
    print('total time - ' + str(data[0]))
    print(path)
    print(' ')


699 W [699]
61 W [699, 61]
151 W [699, 61, 151]
249 W [699, 61, 151, 249]
474 W [699, 61, 151, 249, 474]
986 W [699, 61, 151, 249, 474, 986]
462 W [699, 61, 151, 249, 474, 986, 462]
162 W [699, 61, 151, 249, 474, 986, 462, 162]
150 W [699, 61, 151, 249, 474, 986, 462, 162, 150]
392 W [699, 61, 151, 249, 474, 986, 462, 162, 150, 392]
734 W [699, 61, 151, 249, 474, 986, 462, 162, 150, 392, 734]
699 BR [699, 61, 151, 249, 474, 986, 462, 162, 150, 392, 734]
699 R [61, 151, 249, 474, 986, 462, 162, 150, 392, 734, 699]
392 BR [61, 151, 249, 474, 986, 462, 162, 150, 392, 734, 699]
392 R [61, 151, 249, 474, 986, 462, 162, 150, 734, 699, 392]
699 BR [61, 151, 249, 474, 986, 462, 162, 150, 734, 699, 392]
699 R [61, 151, 249, 474, 986, 462, 162, 150, 734, 392, 699]
392 BR [61, 151, 249, 474, 986, 462, 162, 150, 734, 392, 699]
392 R [61, 151, 249, 474, 986, 462, 162, 150, 734, 392]
392 BR [61, 151, 249, 474, 986, 462, 162, 150, 734, 392]
392 R [61, 151, 249, 474, 986, 462, 162, 150, 734, 392]
699 

KeyError: 151

In [26]:
cache.get_stored_file_list()

AttributeError: 'Cache' object has no attribute 'get_stored_file_list'

In [27]:
sim.log

[[0.7331984002431418,
  699,
  2.0230053879062995,
 [0.7578408929050258,
  61,
  0.1798668126824392,
 [0.7733995095266398,
  151,
  0.1994309607931748,
 [0.7744774068197325,
  249,
  0.4072857771553433,
 [0.8306540865571369,
  474,
  1.3347430052357923,
 [0.8682880353213261,
  986,
  3.8292328050929942,
 [0.9061017879697869,
  462,
  0.054407019579458815,
 [0.9423798758016704,
  162,
  1.2582343419371305,
 [0.9610241044928047,
  150,
  0.44202990216689075,
 [0.9719641253346738,
  392,
  0.08981810484064656,
 [0.996807708183136,
  734,
  0.09407519741054382,
 [8.981810484054442e-05,
  392,
  0.08981810484064656,
 [8.981810484054442e-05,
  392,
  0.08981810484064656,
 [8.981810484054442e-05,
  392,
  0.08981810484064656,
 [8.981810484054442e-05,
  392,
  0.08981810484064656,
 [8.981810484054442e-05,
  392,
  0.08981810484064656,
 [8.981810484054442e-05,
  392,
  0.08981810484064656,
 [9.407519741055914e-05,
  734,
  0.09407519741054382,
 [0.00017986681268244986,
  61,
  0.179866812682439

In [7]:
def sigmoid(x):

    return(1/(1 + np.exp(-x)))

sigmoid(80)

1.0